In [1]:
#copied from https://colab.research.google.com/drive/1pTuQhug6Dhl9XalKB0zUGf4FIdYFlpcX#scrollTo=Z474sSC6oe7A

In [2]:
# import tensorflow as tf

# # Get the GPU device name.
# device_name = tf.test.gpu_device_name()

# # The device name should look like the following:
# if device_name == '/device:GPU:0':
#     print('Found GPU at: {}'.format(device_name))
# else:
#     raise SystemError('GPU device not found')

In [3]:
import torch
# tasks = ['cola', 'MRPC']
TASK = 'cola'


def get_device():
  # If there's a GPU available...
  if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

  # If not...
  else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
  return device

device = get_device()

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2070


In [4]:
!pip install transformers

In [5]:
!pip install wget

In [6]:
import wget
import os

print('Downloading dataset...')

def download_data(task):
  task_to_data = {
    'cola': ('https://nyu-mll.github.io/CoLA/cola_public_1.1.zip', './cola_public_1.1.zip', './cola_public/')
  }
  if task != 'cola':
        return
  url, download_file, unzip_file = task_to_data[task]
  
  # Download the file (if we haven't already)
  if not os.path.exists(download_file):
      wget.download(url, download_file)
      
  # Unzip the dataset (if we haven't already)
  if not os.path.exists(unzip_file):
      !unzip $unzip_file

download_data("cola")



In [7]:
def create_examples(lines, set_type):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, line) in enumerate(lines):
        if i == 0:
            continue
        guid = "%s-%s" % (set_type, i)
        text_a = convert_to_unicode(line[3])
        text_b = convert_to_unicode(line[4])
        if set_type == "test":
            label = "0"
        else:
            label = convert_to_unicode(line[0])
        examples.append(
          {'guid':guid, 'text_a':text_a, 'text_b':text_b, 'label':label})
    return pd.DataFrame(examples)

def read_tsv(input_file, quotechar=None):
    """Reads a tab separated value file."""
    with open(input_file, "r", encoding="utf8") as f:
        reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
        lines = []
        for line in reader:
            lines.append(line)
        return lines
def convert_to_unicode(text):
    if isinstance(text, str):
        return text
    elif isinstance(text, bytes):
        return text.decode("utf-8", "ignore")

In [8]:
import pandas as pd
import csv
def load_data(sample_data = False):
    if TASK == 'cola':
        # Load the dataset into a pandas dataframe.
        df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, 
                                         names=['sentence_source', 'label', 'label_notes', 'sentence'])
    if TASK == 'MRPC':
        return pd.concat((
            [create_examples(read_tsv('glue_data/MRPC/' + name + '.tsv'), name) 
             for name in ['train', 'dev', 'test']]), axis=0)

        # Report the number of sentences.
    print('Number of training sentences: {:,}\n'.format(df.shape[0]))
    
    if sample_data:
        df = df.head(sample_data)
    
    return df

df = load_data()


Number of training sentences: 8,551



In [9]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

C:\Users\leedt\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\leedt\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\leedt\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\leedt\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

Loading BERT tokenizer...


In [10]:
# # max_len = 0

# # # For every sentence...
# # for sent in sentences:

# #     # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
# #     input_ids = tokenizer.encode(sent, add_special_tokens=True)

# #     # Update the maximum sentence length.
# #     max_len = max(max_len, len(input_ids))

# # print('Max sentence length: ', max_len)

# max_len = 30

In [11]:
# def get_features_old(task, df = None):
#   if task == 'cola':
#     return get_features_cola(df)
    
def get_features(df):
    # Get the lists of sentences and their labels.
    if TASK == 'cola':
        sentences = df.sentence.values
    if TASK == 'MRPC':
        sentences = df[['text_a', 'text_b']].values.tolist()
    labels = df.label.values.astype(int)

    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids = []
    attention_masks = []
    token_type_ids = []

    # For every sentence...
    for sent in sentences:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        if TASK == 'MRPC':
            text_b = sent[1]
            sent = sent[0]
        else:
            text_b = None
        encoded_dict = tokenizer.encode_plus(
                            text = sent,
                            text_pair = text_b,
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 64,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt', 
                            truncation = True # Return pytorch tensors.
                       )
        if TASK == 'MRPC':
            token_type_ids.append(encoded_dict['token_type_ids'])

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])
    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    # Print sentence 0, now as a list of IDs.
    print('Original: ', sentences[0])
    print('Token IDs:', input_ids[0])
    ret = [input_ids, attention_masks, labels]
    if TASK == 'MRPC':
        token_type_ids = torch.cat(token_type_ids, dim=0)
        ret += [token_type_ids]
    else:
        ret += [None]
    return ret
  
input_ids, attention_masks, labels, token_type_ids = get_features(df)

Original:  Our friends won't buy this analysis, let alone the next one we propose.
Token IDs: tensor([  101,  2256,  2814,  2180,  1005,  1056,  4965,  2023,  4106,  1010,
         2292,  2894,  1996,  2279,  2028,  2057, 16599,  1012,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


In [12]:
import numpy as np
from torch.utils.data import TensorDataset, random_split
if token_type_ids is not None:
    token_type_ids = [token_type_ids]
else:
    token_type_ids = []

dataset = TensorDataset(input_ids, attention_masks, labels, *token_type_ids)
def get_kfold(input_ids, attention_masks, labels, token_type_ids, k=3):
  dataset = TensorDataset(input_ids, attention_masks, labels, *token_type_ids)
  idx = np.arange(len(input_ids))
  np.random.shuffle(idx)

  fold_length = len(input_ids)//k
  data = []

  for i in range(k):
    start_idx = i*fold_length
    end_idx = (i+1)*fold_length
    if i == k-1:
      end_idx = len(input_ids)

    validation_idx = idx[start_idx: end_idx]
    train_idx = np.concatenate((idx[0: start_idx], idx[end_idx: len(input_ids)]))
    validation_set = TensorDataset(*dataset[validation_idx])
    training_set = TensorDataset(*dataset[train_idx])

    data.append((training_set, validation_set))
  return data
  
data = get_kfold(input_ids, attention_masks, labels, token_type_ids, k=3)
full_dataset = TensorDataset(input_ids, attention_masks, labels, *token_type_ids)

In [13]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import numpy as np
batch_size = 32
from torch import optim
import random
import numpy as np
import pdb
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import datetime
def check_mcc(model, prediction_dataloader):
    predictions, true_labels = make_predictions(model, prediction_dataloader)
    preds = np.concatenate(predictions)
    preds = preds.argmax(1)

    return matthews_corrcoef(df.label, preds)

from sklearn.metrics import matthews_corrcoef
def make_predictions(model, prediction_dataloader):
    # Prediction on test set
    model.eval()
    predictions , true_labels = [], []
    for batch in prediction_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_labels = batch
      with torch.no_grad():
          outputs = model(b_input_ids, token_type_ids=None, 
                          attention_mask=b_input_mask)
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      predictions.append(logits)
      true_labels.append(label_ids)
    model.train()
    return predictions, true_labels

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

def train_model(
    epochs, train_dataloader, validation_dataloader, 
    verbose = False, release = False, prediction_dataloader = None,
    lr = 2e-5, lrbase = 1e-5, lrclass = 1e-4):
    
    
  model = BertForSequenceClassification.from_pretrained(
      "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
      num_labels = 2, 
      output_attentions = False, # Whether the model returns attentions weights.
      output_hidden_states = False, # Whether the model returns all hidden-states.
  )
  mcc_arr = []
  model = model.to(device)
  
  optimizer = AdamW(model.classifier.parameters(),
                    lr = lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                    eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )
#   optimizer = optim.Adam([
# #                 {'params': model.bert.parameters(), 'lr': lrbase},
#                 {'params': model.classifier.parameters(), 'lr': lrclass}],
#                     lr = lr/10, # args.learning_rate - default is 5e-5, our notebook had 2e-5
#                     eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
#                   )
# optim.SGD([
#                 {'params': model.base.parameters()},
#                 {'params': model.classifier.parameters(), 'lr': 1e-3}
#             ], lr=1e-2, momentum=0.9)
  total_steps = len(train_dataloader) * epochs

#   scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                               num_warmup_steps = 0, # Default value in run_glue.py
#                                               num_training_steps = total_steps)
    
#   initval = [v.cpu().detach().numpy() for v in model.parameters()]
  # We'll store a number of quantities such as training and validation loss, 
  # validation accuracy, and timings.
  training_stats = []

  total_t0 = time.time()
  for epoch_i in range(0, epochs):
      if verbose:
          print("")
          print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
          print('Training...')

      t0 = time.time()
      total_train_loss = 0
      model.train()
      for step, batch in enumerate(train_dataloader):
          if step % 40 == 0 and not step == 0:
              elapsed = format_time(time.time() - t0)
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(
                  step, len(train_dataloader), elapsed))
              if release:
                mcc = check_mcc(model, prediction_dataloader)
                print('mcc: ', mcc)
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device).long()
          if len(batch) > 3:
            b_token_type_ids = batch[3].to(device)
          else:
            b_token_type_ids = None
          model.zero_grad()
          loss, logits = model(b_input_ids,  
                               attention_mask=b_input_mask, 
                               labels=b_labels,
                               token_type_ids = b_token_type_ids)
          total_train_loss += loss.item()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#           tempval = [v.cpu().detach().numpy() for v in model.parameters()]
          optimizer.step()
#           scheduler.step()
#           newval = [v.cpu().detach().numpy() for v in model.parameters()]
      avg_train_loss = total_train_loss / len(train_dataloader)
      training_time = format_time(time.time() - t0)

      if verbose:
          print("")
          print("  Average training loss: {0:.2f}".format(avg_train_loss))
          print("  Training epcoh took: {:}".format(training_time))

          print("")
          print("Running Validation...")
      if epoch_i == 0:
        optimizer = AdamW(model.parameters(),
                            lr = lr, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                            eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                          )
#         optimizer = optim.Adam([
#                         {'params': model.bert.parameters(), 'lr': lrbase},
#                         {'params': model.classifier.parameters(), 'lr': lrclass}],
#                             lr = lr/10, # args.learning_rate - default is 5e-5, our notebook had 2e-5
#                             eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
#                           )
      t0 = time.time()
      if release:
        mcc = check_mcc(model, prediction_dataloader)
        print('mcc: ', mcc)
        mcc_arr.append(mcc)
        continue
      model.eval()

      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0

      for batch in validation_dataloader:
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device).long()
          if len(batch) > 3:
            b_token_type_ids = batch[3].to(device)
          else:
            b_token_type_ids = None
          with torch.no_grad():        
              (loss, logits) = model(b_input_ids, 
                                     token_type_ids=b_token_type_ids, 
                                     attention_mask=b_input_mask,
                                     labels=b_labels)
          total_eval_loss += loss.item()
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          total_eval_accuracy += flat_accuracy(logits, label_ids)
      avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
      print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
      avg_val_loss = total_eval_loss / len(validation_dataloader)

      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)

      print("  Validation Loss: {0:.2f}".format(avg_val_loss))
      print("  Validation took: {:}".format(validation_time))

      # Record all statistics from this epoch.
      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Valid. Accur.': avg_val_accuracy,
              'Training Time': training_time,
              'Validation Time': validation_time
          }
      )
  if release:
    return _, _, mcc_arr, model
  print("")
  print("Training complete!")

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
  return avg_val_accuracy, avg_val_loss, training_stats, model

In [14]:


# avg_val_accuracies, avg_val_losses = [], []
# for train_dataset, val_dataset in data:
#   train_dataloader = DataLoader(
#               train_dataset,  # The training samples.
#               sampler = RandomSampler(train_dataset), # Select batches randomly
#               batch_size = batch_size # Trains with this batch size.
#           )
#   validation_dataloader = DataLoader(
#               val_dataset, # The validation samples.
#               sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
#               batch_size = batch_size # Evaluate with this batch size.
#           )


#   epochs = 2
#   avg_val_accuracy, avg_val_loss, training_stats, model = train_model(
#       epochs, train_dataloader, validation_dataloader)
#   avg_val_accuracies.append(avg_val_accuracy)
#   avg_val_losses.append(avg_val_loss)
#   val_loss, val_acc = [sum(arr) / len(arr) for arr in [avg_val_losses, avg_val_accuracies]]


In [15]:
# import pandas as pd

# # Display floats with two decimal places.
# pd.set_option('precision', 2)

# # Create a DataFrame from our training statistics.
# df_stats = pd.DataFrame(data=training_stats)

# # Use the 'epoch' as the row index.
# df_stats = df_stats.set_index('epoch')

# # A hack to force the column headers to wrap.
# #df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# # Display the table.
# df_stats

In [16]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# import seaborn as sns

# # Use plot styling from seaborn.
# sns.set(style='darkgrid')

# # Increase the plot size and font size.
# sns.set(font_scale=1.5)
# plt.rcParams["figure.figsize"] = (12,6)

# # Plot the learning curve.
# plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
# plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# # Label the plot.
# plt.title("Training & Validation Loss")
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.legend()
# plt.xticks([1, 2, 3, 4])

# plt.show()

In [17]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("./cola_public/raw/out_of_domain_dev.tsv", delimiter='\t', 
                 header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

# Create sentence and label lists
if TASK == 'cola':
    sentences = df.sentence.values
if TASK == 'MRPC':
    sentences = df[['texta', 'textb']]
labels = df.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Number of test sentences: 516



Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


## 5.2. Evaluate on Test Set



With the test set prepared, we can apply our fine-tuned model to generate predictions on the test set.

In [18]:
dfhyper = pd.DataFrame()
epochs = 12
mccs = []
full_dataloader = DataLoader(
    full_dataset,  # The training samples.
    sampler = RandomSampler(full_dataset), # Select batches randomly
    batch_size = batch_size # Trains with this batch size.
    )

for lr in [4e-5, 2e-5, 1e-5]:
    _, _, mcc_arr, model = train_model(
      epochs, full_dataloader, None, release = True, lr=lr,
      prediction_dataloader = prediction_dataloader)
    del model
    dfhyper = dfhyper.append({'lr':lr, 'mcc_arr':mcc_arr}, ignore_index=True)
    mccs.append(mcc_arr)
    print(dfhyper)
# for lrbase in [3e-4, 1e-4, 3e-5, 1e-5][::-1]:
#     for lrclass in [3e-4, 1e-4, 3e-5, 1e-5]:
#         epochs = 6
#         _, _, mcc_arr, model = train_model(
#           epochs, full_dataloader, None, release = True, lrclass=lrclass, lrbase=lrbase,
#           prediction_dataloader = prediction_dataloader)
#         del model
#         dfhyper.append({'lrbase':lrbase, 'lrclass':lrclass, 'mcc_arr':mcc_arr}, ignore_index=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  Batch    40  of    268.    Elapsed: 0:00:10.
mcc:  0.10125804187397412
  Batch    80  of    268.    Elapsed: 0:00:20.


C:\Users\leedt\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


mcc:  0.0
  Batch   120  of    268.    Elapsed: 0:00:31.
mcc:  0.0
  Batch   160  of    268.    Elapsed: 0:00:42.
mcc:  0.0
  Batch   200  of    268.    Elapsed: 0:00:53.
mcc:  0.0
  Batch   240  of    268.    Elapsed: 0:01:04.
mcc:  0.0
mcc:  0.0
  Batch    40  of    268.    Elapsed: 0:00:11.
mcc:  0.4260325988448839
  Batch    80  of    268.    Elapsed: 0:00:22.
mcc:  0.47313360836110174
  Batch   120  of    268.    Elapsed: 0:00:34.
mcc:  0.49776042220629524
  Batch   160  of    268.    Elapsed: 0:00:46.
mcc:  0.47594695767828804
  Batch   200  of    268.    Elapsed: 0:00:58.
mcc:  0.4648701710214709
  Batch   240  of    268.    Elapsed: 0:01:10.
mcc:  0.5106784743105157
mcc:  0.3900995794378031
  Batch    40  of    268.    Elapsed: 0:00:11.
mcc:  0.5081700925544508
  Batch    80  of    268.    Elapsed: 0:00:23.
mcc:  0.4821731871156669
  Batch   120  of    268.    Elapsed: 0:00:35.
mcc:  0.38679955868980226
  Batch   160  of    268.    Elapsed: 0:00:47.
mcc:  0.5708818311175128
  B

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

...

mcc:  0.0
  Batch    80  of    268.    Elapsed: 0:00:21.
mcc:  0.0
  Batch   120  of    268.    Elapsed: 0:00:33.
mcc:  0.0
  Batch   160  of    268.    Elapsed: 0:00:44.
mcc:  0.0
  Batch   200  of    268.    Elapsed: 0:00:56.
mcc:  0.0
  Batch   240  of    268.    Elapsed: 0:01:08.
mcc:  0.0
mcc:  0.0
  Batch    40  of    268.    Elapsed: 0:00:11.
mcc:  0.2941151964482428
  Batch    80  of    268.    Elapsed: 0:00:23.
mcc:  0.46356379557216015
  Batch   120  of    268.    Elapsed: 0:00:36.
mcc:  0.5081700925544508
  Batch   160  of    268.    Elapsed: 0:00:48.
mcc:  0.49573583162692847
  Batch   200  of    268.    Elapsed: 0:01:00.
mcc:  0.5503124637396508
  Batch   240  of    268.    Elapsed: 0:01:13.
mcc:  0.40988946267067045
mcc:  0.49238081312278514
  Batch    40  of    268.    Elapsed: 0:00:11.
mcc:  0.5404323554886978
  Batch    80  of    268.    Elapsed: 0:00:23.
mcc:  0.5203129255951798
  Batch   120  of    268.    Elapsed: 0:00:36.
mcc:  0.4913276411363359
  Batch   160  of 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

  Batch    40  of    268.    Elapsed: 0:00:10.


C:\Users\leedt\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


mcc:  0.0
  Batch    80  of    268.    Elapsed: 0:00:21.
mcc:  0.0
  Batch   120  of    268.    Elapsed: 0:00:33.
mcc:  0.0
  Batch   160  of    268.    Elapsed: 0:00:44.
mcc:  0.0
  Batch   200  of    268.    Elapsed: 0:00:56.
mcc:  0.0
  Batch   240  of    268.    Elapsed: 0:01:07.
mcc:  0.0
mcc:  0.0
  Batch    40  of    268.    Elapsed: 0:00:11.
mcc:  0.0
  Batch    80  of    268.    Elapsed: 0:00:23.
mcc:  0.2868657628755636
  Batch   120  of    268.    Elapsed: 0:00:36.
mcc:  0.45514579267616545
  Batch   160  of    268.    Elapsed: 0:00:48.
mcc:  0.429556630789291
  Batch   200  of    268.    Elapsed: 0:01:00.
mcc:  0.44430718683212833
  Batch   240  of    268.    Elapsed: 0:01:13.
mcc:  0.43355530560723166
mcc:  0.46118278685384234
  Batch    40  of    268.    Elapsed: 0:00:11.
mcc:  0.5517111418551632
  Batch    80  of    268.    Elapsed: 0:00:23.
mcc:  0.4708619775979508
  Batch   120  of    268.    Elapsed: 0:00:36.
mcc:  0.5028481961318074
  Batch   160  of    268.    Elaps

In [19]:
[print(f) for f in zip(dfhyper['lr'],dfhyper['mcc_arr'].values)]

(4e-05, [0.0, 0.3900995794378031, 0.5399354485268115, 0.4158380440168314, 0.5101409047574174, 0.42674776695320954, 0.5082697571205093, 0.4665636258184516, 0.519462924306187, 0.5187749525301079, 0.5191916163373586, 0.5657613899464818])
(2e-05, [0.0, 0.49238081312278514, 0.5317491846179545, 0.5508140455151804, 0.49823214802436, 0.534936021898104, 0.5172094006451677, 0.5404323554886978, 0.538903225215818, 0.550529505732458, 0.5154350768151232, 0.5035786192934107])
(1e-05, [0.0, 0.46118278685384234, 0.534579166695121, 0.5187651504700211, 0.5363982246909343, 0.5707238062199783, 0.5611943924327362, 0.5657613899464818, 0.5670493354188807, 0.5858919151990253, 0.5619048392785368, 0.571760808313404])


[None, None, None]

In [20]:
sdfsdf

NameError: name 'sdfsdf' is not defined

In [ ]:
# full_dataloader = DataLoader(
#           full_dataset,  # The training samples.
#           sampler = RandomSampler(full_dataset), # Select batches randomly
#           batch_size = batch_size # Trains with this batch size.
#       )

# epochs = 6
# _, _, mcc_arr, model = train_model(
#   epochs, full_dataloader, None, release = True, lr=1e-4)

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
mcc_arr

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


Let's check out the file sizes, out of curiosity.

In [ ]:
!ls -l --block-size=K ./model_save/

The largest file is the model weights, at around 418 megabytes.

In [ ]:
!ls -l --block-size=M ./model_save/pytorch_model.bin

To save your model across Colab Notebook sessions, download it to your local machine, or ideally copy it to your Google Drive.

In [ ]:
# Mount Google Drive to this Notebook instance.
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Copy the model files to a directory in your Google Drive.
!cp -r ./model_save/ "./drive/Shared drives/ChrisMcCormick.AI/Blog Posts/BERT Fine-Tuning/"

The following functions will load the model back from disk.

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

In [ ]:
# This code is taken from:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L102

# Don't apply weight decay to any parameters whose names include these tokens.
# (Here, the BERT doesn't have `gamma` or `beta` parameters, only `bias` terms)
no_decay = ['bias', 'LayerNorm.weight']

# Separate the `weight` parameters from the `bias` parameters. 
# - For the `weight` parameters, this specifies a 'weight_decay_rate' of 0.01. 
# - For the `bias` parameters, the 'weight_decay_rate' is 0.0. 
optimizer_grouped_parameters = [
    # Filter for all parameters which *don't* include 'bias', 'gamma', 'beta'.
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.1},
    
    # Filter for parameters which *do* include those.
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

# Note - `optimizer_grouped_parameters` only includes the parameter values, not 
# the names.